In [1]:
import numpy as np

There is a question - can we use a laser rangefinder (or something similar) to do fish length detection?

Let's try to answer this with a calibrated laser pair.  This solution uses a pair of parallel laser pointers to project two dots with a nominal separation onto a surface that can then be imaged.  Due to perspective, the corresponding pixel locations of the laser dots changes with the distance to the surface, so we can estimate the distance of the surface.

Let's set the stage for this.  For all simulations, we will use a pinhole camera model.  We'll make life easy and assume an Olympus TG-6 at the widest angle setting, and ignore the air-lens-water interface.

Thus, we have the following parameters:
- Sensor size (width x height): 4000 x 3000
- Pixel Pitch (um): 1.5
- Focal Length (mm): 4.5

Let's place two lasers pointed parallel to the optical axis with a baseline of 50 mm (one 25 mm to the left, one 25 mm to the right).  We will define the global origin on center of the sensor, with $x$ along the width of the sensor, $y$ along the height of the sensor, and $z$ pointing out of the camera into the lens.

In [2]:
sensor_size_px = np.array([4000, 3000])
pixel_pitch_mm = 0.0015
focal_length_mm = 4.5
laser_1_origin = np.array([-0.025, 0, 0])
laser_1_axis = np.array([0, 0, 1])
laser_2_origin = np.array([0.025, 0, 0])
laser_2_axis = np.array([0, 0, 1])

Let's assume we have a flat object 1 m from the sensor.  The lasers will project two dots, 50 mm apart from each other long the y axis.  Specifically, we will have two dots, one at [-0.025, 0, 1], and one at [0.025, 0, 1].

In [3]:
plane_normal = np.array([0, 0, 1])
plane_origin = np.array([0, 0, 1])

For a plane defined as $\left<\left(\mathbf{p} - \mathbf{p_0}\right), \mathbf{n}\right> = 0$ and a vector defined as $\mathbf{p} = \mathbf{l_0} + \mathbf{l}d, d \in \mathbb{R}$, the point of intersection is the point on the vector such that $d = \frac{\left<\left(\mathbf{p_0} - \mathbf{l_0}\right), \mathbf{n}\right>}{\left<\mathbf{l}, \mathbf{n}\right>}$

In [4]:
laser_1_scalar = np.dot((plane_origin - laser_1_origin), plane_normal) / np.dot(laser_1_axis, plane_normal)
laser_1_dot = laser_1_origin + laser_1_axis * laser_1_scalar
laser_2_scalar = np.dot((plane_origin - laser_2_origin), plane_normal) / np.dot(laser_2_axis, plane_normal)
laser_2_dot = laser_2_origin + laser_2_axis * laser_2_scalar
laser_1_dot, laser_2_dot

(array([-0.025,  0.   ,  1.   ]), array([0.025, 0.   , 1.   ]))

Any point $[x_1, x_2, x_3]$ in the world in the field of view will project onto the image plane as follows:
$$ \begin{pmatrix}y_1\\y_2\end{pmatrix} = -\frac{f}{x_3}\begin{pmatrix}x_1\\x_2\end{pmatrix} $$

In [5]:
laser_1_projection = -focal_length_mm / 1e3 / laser_1_dot[2] * laser_1_dot[0:2] / (pixel_pitch_mm / 1e3) # in pixels
laser_2_projection = -focal_length_mm / 1e3 / laser_2_dot[2] * laser_2_dot[0:2] / (pixel_pitch_mm / 1e3) # in pixels
laser_1_projection, laser_2_projection

(array([75., -0.]), array([-75.,  -0.]))